# Git - Control de Versiones
*Sofía García Salas*

*2021-03-02*

***

Estas son mis notas sobre la clase de MIT [The Missing Semester](https://missing.csail.mit.edu/2020/version-control/).

## Control de versiones
`Git` es un sistema de control de versiones que permite documentar los cambios a código u otro documentos y folders. También facilitan el trabajo en equipo.

Estos sistemas almacenan la historia de cambios tomando una fotografía al estado de los archivos y folders. No solo tienen la historia de los cambios a estos archivos sino que también almacenan metadata (e.g. quién realizó un cambio, en qué momento).

### Utilidad
Sirve no solo en proyectos grandes, sino en proyecto personales para poder revisar versiones antiguos del código, ver por qué se cambio algo (viendo commit messages), trabajar en paralelo con distintas branches para el desarrollo, etc. 

### Git
Se ha vuelto el sistema de control de versiones más popular.

Se interfaz no es muy amigable con el usuario, por lo que el profesor recomienda empezar aprendiendo su diseño e ideas. Al entender esto, será más fácil utilizar la interfaz y resolver problemas.

## Modelo de datos de Git
### Modelo de folders y archivos
Es una colección de archivos y folders dentro de un directorio maestro.
Un folder es denominado como `tree` y un archivo como `blob`.

```
root
|-foo (tree)
| |-bar.txt (blob)
|-baz.txt

```

El folder `root`es el directorio al que se le está dando seguimiento.

### Modelo de historia
Una forma de crear un modelo de historia es tomar una fotografía de los folders y archivos en el directorio. La historia se vuelve una secuencia lineal de estas fotografías. Es muy similar a tener copias de folders con marcas de tiempo.

Git utilizado un modelo más complejo llamado **Directed acyclic graph**. 

Con este modelo uno puede separar versiones (`fork`) para trabajar en ramas (`branches`) en paralelo sin interferencia; por ejemplo, trabajar es una nueva característica y en paralelo arreglar un desperfecto en el código en producción. 
Estas ramas después se pueden unir (`merge`) y la nueva fotografía tendrá como referencia las dos versiones que se unieron.



![history](img/history.jpg)

Es pseudocódigo, el modelo de archivos e historia de `Git`se puede explicar de la siguiente manera:

**Tipos de estructuras:**
```
type blob = array<byte>    # file = coleccion de bytes

type tree = map<string, tree | blob> # folder = mapea un nombre de directorio a contenido (archivos o folders)

type commit = struct{           # metadata con la información de la fotografía
    parents: array<commit>
    author: string
    message: string
    snapshot: tree  # esta es una referencia al objeto no el objeto en si
```
**Almacenamiento en disco:**los objetos son los que Git almacena en disco
```
type object = blog | tree | commit

objects = map<string, objects>

# para almacenar en disco se utiliza el hash del objeto
def store(o)
    id = sha1(o) # hash del objeto
    objects[id] = o

}

df load(id)
    return objects[id]
```
**Referencias**:
mapea referencias de un nombre legible para el humano con su hash respectivo
```
references = map<string, string>
```

La historia y los hash son inmutables, pero las referencias sí se pueden cambiar.
En forma general, todos los comandos de `Git` son manipulaciones de los datos de los *objetos* o de las *referencia*

##